<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

In [2]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'

response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [3]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [4]:
raw_data[:10]

'emma\nolivi'

In [5]:
dataset = raw_data.split("\n")

In [6]:
dataset[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [7]:
vocab = set(raw_data)

In [8]:
vocab_len = len(vocab)

In [9]:
vocab_len

27

In [10]:
stoi = {char:i for i, char in enumerate(vocab)}

In [11]:
itos = {i:char for i, char in enumerate(vocab)}

In [12]:
import torch

In [13]:
import torch.nn.functional as functional

In [14]:
# make dataset
xs, ys = [], []
for data in dataset:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

264094

In [15]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([54, 27])

In [16]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([-1.1150, -2.1575, -1.3457,  0.3497, -2.6369,  1.0399,  2.0206,  2.9631,
         3.3517,  1.9344, -0.3824,  1.3967, -0.9577,  0.6481,  0.3936,  2.1484,
         2.2706,  1.4456,  0.2828, -0.9791,  0.0200,  0.8186, -0.7874,  0.5812,
         1.6020, -0.1085,  1.2318], grad_fn=<SqueezeBackward4>)

In [17]:
(xenc @ ws).shape

torch.Size([27])

In [18]:
num_epochs = 100
lr = 50

In [19]:
torch.randn((3, 1))

tensor([[-1.0210],
        [ 0.4341],
        [-0.4939]])

In [20]:
for epoch in range(num_epochs):
    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

4.131076335906982
3.416553497314453
3.092968225479126
2.9063868522644043
2.7875494956970215
2.7067811489105225
2.648846387863159
2.6051576137542725
2.5709540843963623
2.5434374809265137
2.520812511444092
2.501863718032837
2.4857404232025146
2.471829891204834
2.459683418273926
2.4489684104919434
2.43943452835083
2.430889844894409
2.423185348510742
2.4162020683288574
2.4098429679870605
2.40402889251709
2.3986928462982178
2.3937795162200928
2.3892407417297363
2.3850364685058594
2.381131410598755
2.377495050430298
2.374101161956787
2.3709263801574707
2.367949962615967
2.3651552200317383
2.3625245094299316
2.360044479370117
2.3577022552490234
2.3554866313934326
2.3533878326416016
2.351395845413208
2.3495030403137207
2.3477022647857666
2.3459861278533936
2.3443496227264404
2.3427860736846924
2.341291904449463
2.3398613929748535
2.3384907245635986
2.337177038192749
2.3359155654907227
2.3347036838531494
2.3335390090942383
2.3324179649353027
2.33133864402771
2.330298662185669
2.3292956352233887

In [21]:
xs[:10]

tensor([[18,  4],
        [ 4,  4],
        [ 2, 26],
        [26,  0],
        [ 0,  1],
        [ 1,  0],
        [23,  1],
        [ 0, 15],
        [15, 23],
        [23,  7]])

In [22]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

'Romeeaeiieaeaarikeamimosaoueaurtaymaaiaaioataabneeiaoaeaaooaiiaiiaaaaoiaimbaaiaeaauibaiieloieaaiiaiueemidooiaoygaiekeaeeaieaaiiaaaisioaiaiiaiwoaimoeioaiyiaeaiaauaijiymeiiooaiabyeasbaiearaamyaaaarapaoiiaoaiiiameaooyybeieiibaaeeeagaeeeeeieaokreasaascabeeebeseoiidaaayseeaiuaaeeaasemaaaamyabiyaiiieaiiaeadbariieaaieaybieeaoyeieaeyeaiaiireiaiiaie\niaaioamebairiikeauradiapoieiumeoieoaaaaeuiideamataeraiayooeiieeajoieybeiyaeiiamiubaaaabdaiomaemuerimieiapaiaqeaiyirieioaaaiaiiiaauyyaauaamu\niaaaaiairaiaeaiaatix'